In [2]:
import cv2
import os
import pandas as pd
import ast

In [22]:
!cd '\Файлы и программы\Учеба\ВКР\Dataset'

C:\Файлы и программы\Учеба\ВКР\Dataset


In [23]:
data_bbox = pd.read_csv('bbox.csv')
data_bbox

,filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes
0,1_original.jpg,398745,{},1,0,"{""name"":""rect"",""x"":1123,""y"":775,""width"":134,""h...",{}
1,2_original.jpg,172977,{},1,0,"{""name"":""rect"",""x"":520,""y"":414,""width"":564,""he...",{}
2,3_original.jpg,144855,{},1,0,"{""name"":""rect"",""x"":442,""y"":236,""width"":370,""he...",{}
3,4_original.jpg,205292,{},1,0,"{""name"":""rect"",""x"":517,""y"":411,""width"":404,""he...",{}
4,5_original.jpg,118295,{},1,0,"{""name"":""rect"",""x"":340,""y"":227,""width"":470,""he...",{}
...,...,...,...,...,...,...,...
600,N286_286_original.jpg,29934,{},1,0,"{""name"":""rect"",""x"":25,""y"":161,""width"":414,""hei...",{}
601,N287_287_original.jpg,32886,{},1,0,"{""name"":""rect"",""x"":30,""y"":178,""width"":416,""hei...",{}
602,N288_288_original.jpg,30154,{},1,0,"{""name"":""rect"",""x"":54,""y"":178,""width"":395,""hei...",{}
603,N289_289_original.jpg,24162,{},1,0,"{""name"":""rect"",""x"":23,""y"":142,""width"":430,""hei...",{}


In [12]:
def xywh_to_xyxy(x_min, y_min, box_width, box_height):
    x_max = x_min + box_width
    y_max = y_min + box_height
    return x_min, y_min, x_max, y_max

def xywh_to_xywh_scaled(x_min, y_min, box_height, box_width, image_width, image_height):
    x_center = x_min + box_width // 2
    y_center = y_min + box_height // 2
    return x_center / image_width, y_center / image_height, box_width / image_width, box_height / image_height

In [41]:
class_num = 0

for i, row in data_bbox.iterrows():
    
    annotation = ast.literal_eval(
        row['region_shape_attributes']
    )
    

    filename = row['filename'].split('.')[0]
    image = cv2.imread(os.path.join('yolov4', 'images', row['filename']))

    width, height = None, None
    try:
        width, height = image.shape[1], image.shape[0]
    except AttributeError:
        print(row)
        pass

    x, y, w, h = xywh_to_xywh_scaled(annotation['x'], annotation['y'], annotation['width'], annotation['height'],
                                             width, height)
    
    
    if os.path.exists(f'yolov4/images/{filename}.txt'):
        mode = 'a'
        line = f'\n{class_num} {x} {y} {w} {h}'
    else:
        mode = 'w'
        line = f'{class_num} {x} {y} {w} {h}'
        
    with open(f'yolov4/images/{filename}.txt', mode) as file:
        file.write(line)
        

filename                                                 N28_28_original.jpg
file_size                                                             191090
file_attributes                                                           {}
region_count                                                               1
region_id                                                                  0
region_shape_attributes    {"name":"rect","x":47,"y":151,"width":383,"hei...
region_attributes                                                         {}
Name: 342, dtype: object


# Create Train and validation Dataset

In [7]:
import shutil

dataframe = pd.read_csv('../Data/Classification/city_year_train.csv')
for _, row in dataframe.iterrows():
    image_name = row['ImageName']
    shutil.copy(
        os.path.join('../Data/Detection/yolov4/obj', image_name),
        os.path.join('../Data/Detection/yolov4/val', image_name)
    )

    shutil.copy(
        os.path.join('../Data/Detection/yolov4/obj', image_name.split('.')[0] + '.txt'),
        os.path.join('../Data/Detection/yolov4/val', image_name.split('.')[0] + '.txt')
    )
